In [ ]:
import pandas as pd
import param
import numpy as np
import holoviews as hv
from holoviews import opts
from functools import partial
hv.extension('bokeh')

# Streams as Part of a Workflow

In [ ]:
data_file = '../data/iris.data'
colnames = [
    'sepal length',
    'sepal width',
    'petal length',
    'petal width',
    'species'
]
iris = pd.read_csv(data_file, names=colnames, index_col=False)
iris.head()

In [ ]:
# Get ranges for each numeric column
istat = iris.describe()
iranges = istat.apply(lambda col: (col['min'], col['max'])).to_dict()

In [ ]:
# Compose main plot
sepals = hv.Scatter(
    iris,
    kdims='sepal length',
    vdims=[
        'sepal width',
        'species'
    ])
# Stream selection events from main plot
selection = hv.streams.Selection1D(source=sepals)

# Function to plot a histogram
def plothist(index, col):
    if index == []:
        index = iris.index.tolist()
    freq, edges = np.histogram(iris.loc[index, col], 20)
    return hv.Histogram(
        (edges, freq),
        kdims=hv.Dimension(
            'x',
            label=col,
            range=iranges[col])).relabel(col)

# DynamicMap histogram plots using partials to specify column
# Extra redim and norm to cause plots to autoscale with new data
hist_list = []
for col in ['sepal length', 'sepal width', 'petal length', 'petal width']:
    hist_list.append(hv.DynamicMap(
        partial(plothist, col=col),
        streams=[selection])\
            .redim.range()\
            .opts(norm=dict(framewise=True)))

# compose layout of 4 responsive plots
layout = hv.Layout(hist_list).cols(2)

### Make Selections

In [ ]:
# Main Plot
sepals.opts(color='species', cmap='Category10', size=6, width=400, height=400, tools=['box_select'])

### Explore Selected Data

In [ ]:
layout

### Continue Analysis with Selected Data

In [ ]:
iris.loc[selection.index]